In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# Função para extrair o título do produto
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'}).text.strip()
    except AttributeError:
        title = ""
    return title

# Função para extrair o preço do produto
def get_price(soup):
    try:
        price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip()
    except AttributeError:
        price = ""
    return price

# Função para extrair a classificação do produto
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={'class': 'a-icon-alt'}).text.strip()
    except AttributeError:
        rating = ""
    return rating

# Função para extrair o número de avaliações dos usuários
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).text.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Função para extrair o status de disponibilidade
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'}).find("span").text.strip()
    except AttributeError:
        available = "Not Available"
    return available

def get_product_details(link):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(link)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "productTitle")))
    time.sleep(random.uniform(1, 2))  # Atraso randômico entre 1 a 2 segundos
    new_webpage = driver.page_source
    driver.quit()
    new_soup = BeautifulSoup(new_webpage, "html.parser")
    return {
        'title': get_title(new_soup),
        'price': get_price(new_soup),
        'rating': get_rating(new_soup),
        'reviews': get_review_count(new_soup),
        'availability': get_availability(new_soup)
    }

if __name__ == '__main__':
    produto = input("Digite o nome do produto: ").strip().replace(" ", "+")
    
    # Configurar o WebDriver do Chrome
    options = Options()
    options.add_argument("--headless")  # Executar em modo headless (sem abrir uma janela do navegador)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # URL do site
    URL = "https://www.amazon.com.br/s?k=" + produto
    
    # Acessar a página com o WebDriver
    driver.get(URL)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.a-link-normal.s-no-outline")))
    
    # Obter o HTML da página
    webpage = driver.page_source
    soup = BeautifulSoup(webpage, "html.parser")
    
    # Buscar links dos produtos (limitar a 20 produtos)
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})[:20]
    links_list = ["https://www.amazon.com.br" + link.get('href') for link in links]
    
    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}
    
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(get_product_details, link) for link in links_list]
        for future in as_completed(futures):
            product_details = future.result()
            d['title'].append(product_details['title'])
            d['price'].append(product_details['price'])
            d['rating'].append(product_details['rating'])
            d['reviews'].append(product_details['reviews'])
            d['availability'].append(product_details['availability'])
    
    # Fechar o WebDriver
    driver.quit()
    
    # Criar um DataFrame a partir do dicionário
    amazon_df = pd.DataFrame.from_dict(d)
    
    # Substituir títulos vazios por NaN e remover linhas com títulos NaN
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['title'])
    
    # Filtrar apenas preços numéricos e converter para float
    amazon_df['price'] = amazon_df['price'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['price'])
    amazon_df = amazon_df[amazon_df['price'].str.contains('R\$')]
    amazon_df['price_float'] = amazon_df['price'].str.replace('R$', '').str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    
    filtro = input('\nDeseja adicionar o filtro de produtos? (sim/não): ').strip().lower()
    if filtro == 'sim':
        # Encontrar o preço máximo
        preco_maximo = amazon_df['price_float'].max()

        # Definir o limiar de 50% abaixo do preço máximo
        limiar = preco_maximo * 0.50

        # Filtrar produtos com preço entre o limiar e o preço máximo
        produtos_filtrados = amazon_df[(amazon_df['price_float'] >= limiar) & (amazon_df['price_float'] <= preco_maximo)]
        # Ordenar os produtos filtrados em ordem crescente de preço        
        produtos_filtrados = produtos_filtrados.sort_values(by='price_float')
    else:
        # Caso deseje mostrar todos os produtos e não só aqueles entre o preço max e 50% do preço max
        produtos_filtrados = amazon_df.sort_values(by='price_float')
    
    # Adicionar uma nova coluna 'Número' para a numeração
    produtos_filtrados.insert(0, 'Número', range(1, len(produtos_filtrados) + 1))
    
    # Remover a coluna 'price_float'
    produtos_filtrados = produtos_filtrados.drop(columns=['price_float'])
    
    # Renomear as colunas para português
    produtos_filtrados.columns = ['Ordem', 'Título', 'Preço', 'Classificação', 'Avaliações', 'Disponibilidade']
    
    # Centralizar o texto das colunas
    produtos_filtrados_style = produtos_filtrados.style.set_properties(**{'text-align': 'center'})
    produtos_filtrados_style = produtos_filtrados_style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

    display(produtos_filtrados_style.hide(axis="index"))
    
    # Salvar o DataFrame filtrado em um arquivo Excel/CSV/JSON
    produtos_filtrados.to_excel("amazon_produtos_filtrados.xlsx", header=True, index=False)
    produtos_filtrados.to_csv("amazon_produtos_filtrados.csv", header=True, index=False)
    produtos_filtrados.to_json("amazon_produtos_filtrados.json", index=False)


Ordem,Título,Preço,Classificação,Avaliações,Disponibilidade
1,Fonte De Energia Bivolt 100~240v Nintendo Wii Ac Adaptador,"R$33,89","4,5 de 5 estrelas",168 avaliações de clientes,Em estoque
2,Wii2hdmi - Adaptador Conversor Hdmi Para Wii Full Hd Tv Lcd,"R$39,90","4,6 de 5 estrelas",351 avaliações de clientes,Em estoque
3,Barra Sensor Infravermelho Para Nintendo Wii E Wii U,"R$39,90","4,8 de 5 estrelas",28 avaliações de clientes,Em estoque
4,Wii2hdmi - Adaptador Conversor Nintendo Wii Para Hdmi 1080p,"R$49,90","4,4 de 5 estrelas",50 avaliações de clientes,Em estoque
5,Controle Clássico Classic Grip Nintendo Wii e Wii U Branco,"R$94,99","4,6 de 5 estrelas",11 avaliações de clientes,Em estoque
6,"PORTHOLIC Conversor Wii para HDMI 1080P para dispositivo Full HD, adaptador Wii HDMI com conector de áudio de 3,5 mm e saída HDMI compatível com Wii, Wii U, HDTV, monitor - suporta todos os modos de","R$112,74","4,5 de 5 estrelas",16.875 avaliações de clientes,Em estoque
7,"AUTOUTLET Conversor Wii para HDMI 1080P com cabo HDMI de alta velocidade de 1,8 m, adaptador HDMI Wii2 com indicador azul, saída de vídeo e áudio com conector de áudio de 3,5 mm compatível com","R$129,79","4,5 de 5 estrelas",728 avaliações de clientes,Em estoque
8,"LaiChauKit Wii Remote with Nunchuck, Compatible with Nintendo Wii/Wii U, Wii Controller with Nunchuck, with Silicone Case and Wrist Strap","R$188,96","4,3 de 5 estrelas",182 avaliações de clientes,Em estoque
9,TechKen Estação de carregamento para controle Wii com 4 baterias recarregáveis de 2800 mAh e cabo USB compatível com controles remotos Nintendo Wii,"R$208,26","4,2 de 5 estrelas",185 avaliações de clientes,Em estoque
10,Mini Console Retro Super 3D Games com 130 mil jogos + 2 Controles - Super 3D Games,"R$299,00","4,3 de 5 estrelas",213 avaliações de clientes,Em estoque
